# Import and Initialize

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation
from nilmtk.utils import print_dict
from nilmtk.metrics import f1_score, rms_error_power, mean_normalized_error_power
from nilmtk.legacy.disaggregate.hart_85 import Hart85

%matplotlib inline
rcParams['figure.figsize'] = (13, 6)

# Load Data

In [80]:
test_data = DataSet('../SeniorDataset/h5_files/96Hour_30Second.h5')
train_data = DataSet('../SeniorDataset/h5_files/96Hour_30Second.h5')

# train_data.set_window(start='2022-01-01T00', end='2022-01-02T23') #48 hr
# train_data.set_window(start='2022-01-01T00', end='2022-01-02T00') #24 hr
# train_data.set_window(start='2022-01-02T00', end='2022-01-02T12') #12 hr
# train_data.set_window(start='2022-01-01T08', end='2022-01-01T12') #4 hr
# train_data.set_window(start='2022-01-02T00', end='2022-01-02T01') #1 hr
# train_data.set_window(start='2022-01-01T0430', end='2022-01-01T0500') #30 min

train_elec = train_data.buildings[1].elec
train_mains = train_elec.mains()
train_submeters = train_elec.submeters()

# test_data.set_window(start='2022-01-03T00', end='2022-01-04T23') #48 hr
# test_data.set_window(start='2022-01-03T00', end='2022-01-04T00') #24 hr
# test_data.set_window(start='2022-01-04T00', end='2022-01-04T12') #12 hr
# test_data.set_window(start='2022-01-03T08', end='2022-01-03T12') #4 hr
# test_data.set_window(start='2022-01-03T08', end='2022-01-03T09') #1 hr
# test_data.set_window(start='2022-01-03T0230', end='2022-01-03T0300') #30 min

test_elec = test_data.buildings[1].elec
test_mains = test_elec.mains()
test_submeters = test_elec.submeters()

h = Hart85()

# Train Hart85 Model

In [ ]:
h.train(train_mains, columns=[('power', 'apparent')], buffer_size=100)

# Disaggregate

In [67]:
filename = 'output6.h5'

In [ ]:
output = HDFDataStore(filename, 'w')
df = h.disaggregate(test_mains, output, sample_period=60)
output.close()

# Visualize the Data

In [ ]:
#Hart is unsupervised
h.best_matched_appliance(test_submeters, df)
instance_dict = {2: 'fridge', 3: 'kettle', 4: 'microwave', 5: 'toaster', 6: 'hair dryer', 7: 'slow cooker'}

In [ ]:
instance = 2

df_index = 0
df_pred = next(test_elec[instance_dict[instance], 1].load())
merged_df = pd.merge(df[df_index], df_pred, left_index=True, right_index=True)
merged_df[df_index].plot(c='r')
merged_df['power', 'apparent'].plot()
plt.legend(["Predicted", "Ground truth"])
plt.ylabel("Power (W)")
plt.xlabel("Time")

In [ ]:
instance = 7

df_index = 0
df_pred = next(test_elec[instance_dict[instance], 1].load())
merged_df = pd.merge(df[df_index], df_pred, left_index=True, right_index=True)
merged_df[df_index].plot(c='r')
merged_df['power', 'apparent'].plot()
plt.legend(["Predicted", "Ground truth"])
plt.ylabel("Power (W)")
plt.xlabel("Time")

# Statistical Performance Metrics

In [ ]:
prediction_data = DataSet(filename)

predictions_metergroup = prediction_data.buildings[1].elec
ground_truth_metergroup = test_elec

# print(predictions_metergroup)
# print(ground_truth_metergroup)

f1 = f1_score(predictions_metergroup, ground_truth_metergroup)
mnp = mean_normalized_error_power(predictions_metergroup, ground_truth_metergroup)
rmse = rms_error_power(predictions_metergroup, ground_truth_metergroup)

In [ ]:
print('F1:')
print(f1)

In [ ]:
print("Mean Normalized Error Power:")
print(mnp)
print("RMSE:")
print(rmse)